<div class="alert alert-block alert-info">
<b>Deadline:</b> March 29, 2023 (Wednesday) 23:00
</div>

# Exercise 1. Sequence-to-sequence modeling with recurrent neural networks

The goals of this exercise are
* to get familiar with recurrent neural networks used for sequential data processing
* to get familiar with the sequence-to-sequence model for machine translation
* to learn PyTorch tools for batch processing of sequences with varying lengths
* to learn how to write a custom `DataLoader`

You may find it useful to look at this tutorial:
* [Translation with a Sequence to Sequence Network and Attention](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)

In [14]:
skip_training = True  # Set this flag to True before validation and submission

In [15]:
# During evaluation, this cell sets skip_training to True
# skip_training = True

import tools, warnings
warnings.showwarning = tools.customwarn

In [16]:
import os
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader

import tools
import tests

In [17]:
# When running on your own computer, you can specify the data directory by:
# data_dir = tools.select_data_dir('/your/local/data/directory')
data_dir = tools.select_data_dir()

The data directory is /coursedata


In [18]:
# Select the device for training (use GPU if you have one)
#device = torch.device('cuda:0')
device = torch.device('cpu')

In [19]:
if skip_training:
    # The models are always evaluated on CPU
    device = torch.device("cpu")

## Data

The dataset that we are going to use consists of pairs of sentences in French and English.

In [20]:
from data import TranslationDataset, MAX_LENGTH, SOS_token, EOS_token

trainset = TranslationDataset(data_dir, train=True)

* `TranslationDataset` supports indexing as required by `torch.utils.data.Dataset`.
* Sentences are tensors of maximum length `MAX_LENGTH`.
* Words in a (sentence) tensor are represented as an index (integer) in a language vocabulary.
* The string representation of a word from the source language can be obtained from index `i` with `dataset.input_lang.index2word[i]`.
* Similarly for the target language `dataset.output_lang.index2word[j]`.

Let us look at samples from that dataset.

In [21]:
src_sentence, tgt_sentence = trainset[np.random.choice(len(trainset))]
print('Source sentence: "%s"' % ' '.join(trainset.input_lang.index2word[i.item()] for i in src_sentence))
print('Sentence as tensor of word indices:')
print(src_sentence)

print('Target sentence: "%s"' % ' '.join(trainset.output_lang.index2word[i.item()] for i in tgt_sentence))
print('Sentence as tensor of word indices:')
print(tgt_sentence)

Source sentence: "il se plaint tout le temps . EOS"
Sentence as tensor of word indices:
tensor([  24,  533, 3048,   35,  454, 1423,    5,    1])
Target sentence: "he is always complaining . EOS"
Sentence as tensor of word indices:
tensor([  14,   40,  778, 1246,    4,    1])


In [22]:
print('Number of source-target pairs in the training set: ', len(trainset))

Number of source-target pairs in the training set:  8682


## Sequence-to-sequence model for machine translation

In this exercise, we are going to build a machine translation system which transforms a sentence in one language into a sentence in another one. The computational graph of the translation model is shown below:

<img src="seq2seq.png" width=900>

We are going to use a simplified model without the dotted connections.

## Custom DataLoader

We would like to train the sequence-to-sequence model using mini-batch training.
One difficulty of mini-batch training in this case is that sequences may have varying lengths and this has to be taken into account when building the computational graph. Luckily, PyTorch has tools to support batch processing of such sequences.
To use those tools, we need to write a custom data loader which puts sequences of varying lengths in the same tensor. We can customize the data loader by providing a custom `collate_fn` as explained [here](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader).

Our collate function:
- combines sequences from the source language in a single tensor with extra values (at the end) filled with `PADDING_VALUE=0`.
- combines sequences from the target language in a single tensor with extra values (at the end) filled with `PADDING_VALUE=0`.

**Important**:
- Later in the code (not in this `collate` function), we will convert source sequences to objects of class [PackedSequence](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.PackedSequence.html#torch.nn.utils.rnn.PackedSequence) which can be processed by recurrent units such as `GRU` or `LSTM`. Conversion to `PackedSequence` objects requires sequences to be sorted by their lengths.
**Therefore, the returned source sequences should be sorted by length in a decreasing order.**
* The target sequences need not be sorted by their lengths because we have to keep the same order of sequences in the source and target tensors.

Your task is to implement the collate function.

In [23]:
PADDING_VALUE = 0

In [24]:
from torch.nn.utils.rnn import pad_sequence

def collate(list_of_samples):
    """Merges a list of samples to form a mini-batch.

    Args:
      list_of_samples is a list of tuples (src_seq, tgt_seq):
          src_seq is of shape (src_seq_length,)
          tgt_seq is of shape (tgt_seq_length,)

    Returns:
      src_seqs of shape (max_src_seq_length, batch_size): Tensor of padded source sequences.
          The sequences should be sorted by length in a decreasing order, that is src_seqs[:,0] should be
          the longest sequence, and src_seqs[:,-1] should be the shortest.
      src_seq_lengths: List of lengths of source sequences.
      tgt_seqs of shape (max_tgt_seq_length, batch_size): Tensor of padded target sequences.
    """
    # YOUR CODE HERE
    #raise NotImplementedError()
        
    # The returned source sequences should be sorted by length in a decreasing order
    # The target sequences need not be sorted by their lengths because we have to keep the same 
    # order of sequences in the source and target tensors.
    
    sorted_by_src_list = sorted(list_of_samples, key=lambda sequence: sequence[0].size(), reverse=True)
    
    # The separated source and target sequences
    src_seqs = [sample[0] for sample in sorted_by_src_list]
    tgt_seqs = [sample[1] for sample in sorted_by_src_list]
    
    # List of lengths of source sequences
    src_seq_lengths = [len(src) for src in src_seqs]
    
    # Our collate function:
    # - combines sequences from the source language in a single tensor with extra values 
    # (at the end) filled with `PADDING_VALUE=0`.
    # - combines sequences from the target language in a single tensor with extra values 
    # (at the end) filled with `PADDING_VALUE=0`.
    src_seqs = pad_sequence(src_seqs)
    tgt_seqs = pad_sequence(tgt_seqs)
    
    return src_seqs, src_seq_lengths, tgt_seqs

In [25]:
def test_collate_shapes():
    pairs = [
        (torch.LongTensor([1, 2]), torch.LongTensor([3, 4, 5])),
        (torch.LongTensor([6, 7, 8]), torch.LongTensor([9, 10])),
    ]
    pad_src_seqs, src_seq_lengths, pad_tgt_seqs = collate(pairs)
    assert type(src_seq_lengths) == list, "src_seq_lengths should be a list."
    assert pad_src_seqs.shape == torch.Size([3, 2]), f"Bad pad_src_seqs.shape: {pad_src_seqs.shape}"
    assert pad_src_seqs.dtype == torch.long
    assert pad_tgt_seqs.shape == torch.Size([3, 2]), f"Bad pad_tgt_seqs.shape: {pad_tgt_seqs.shape}"
    assert pad_tgt_seqs.dtype == torch.long
    print('Success')

test_collate_shapes()

Success


In [86]:
# This cell tests collate() function

def test_collate_fn():
    pairs = [
        (torch.tensor([1, 2]), torch.tensor([3, 4, 5])),
        (torch.tensor([6, 7, 8]), torch.tensor([9, 10])),
        (torch.tensor([11, 12, 13, 14]), torch.tensor([15])),
    ]
    pad_src_seqs, src_seq_lengths, pad_tgt_seqs = collate(pairs)
    assert pad_src_seqs.shape == torch.Size([4, 3]), f"Bad pad_src_seqs.shape: {pad_src_seqs.shape}"
    assert pad_tgt_seqs.shape == torch.Size([3, 3]), f"Bad pad_tgt_seqs.shape: {pad_tgt_seqs.shape}"
    print('Source sequences combined:')
    print(pad_src_seqs)
    expected = torch.tensor([
      [11, 6, 1],
      [12, 7, 2],
      [13, 8, 0],
      [14, 0, 0],
    ])
    assert (pad_src_seqs == expected).all(), "pad_src_seqs does not match expected values"

    print(src_seq_lengths)
    if isinstance(src_seq_lengths[0], torch.Size):
        src_seq_lengths = sum((list(l) for l in src_seq_lengths), [])
    else:
        src_seq_lengths = [int(l) for l in src_seq_lengths]
    assert src_seq_lengths == [4, 3, 2], f"Bad src_seq_lengths: {src_seq_lengths}"

    print('Target sequences combined:')
    print(pad_tgt_seqs)
    expected = torch.tensor([
      [15,  9, 3],
      [ 0, 10, 4],
      [ 0,  0, 5],
    ])
    assert (pad_tgt_seqs == expected).all(), "pad_tgt_seqs0 does not match expected values"
    print('Success')

test_collate_fn()


Source sequences combined:
tensor([[11,  6,  1],
        [12,  7,  2],
        [13,  8,  0],
        [14,  0,  0]])
[4, 3, 2]
Target sequences combined:
tensor([[15,  9,  3],
        [ 0, 10,  4],
        [ 0,  0,  5]])
Success


In [87]:
# We create custom DataLoader using the implemented collate function
# We are going to process 64 sequences at the same time (batch_size=64)
trainloader = DataLoader(dataset=trainset, batch_size=64, shuffle=True, collate_fn=collate, pin_memory=True)

## Encoder

The encoder encodes a source sequence $(x_1, x_2, ..., x_T)$ into a single vector $h_T$ using the following recursion:
$$
  h_{t} = f(h_{t-1}, x_t) \qquad t = 1, \ldots, T
$$
where:
* intial state $h_0$ is often chosen arbitrarily (we choose it to be zero)
* function $f$ is defined by the type of the RNN cell (in our experiments, we will use [GRU](https://pytorch.org/docs/stable/generated/torch.nn.GRU.html#torch.nn.GRU))
* $x_t$ is a vector that represents the $t$-th word in the source sentence.

A common practice in natural language processing is to _learn_ the word representations $x_t$ (instead of, for example, using one-hot coded vectors). In PyTorch, this is supported by class [Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html#torch.nn.Embedding) which we are going to use.

The computational graph of the encoder is shown below:

<img src="seq2seq_encoder.png" width=500>

Your task is to implement the `forward` function of the encoder. It should contain the following steps:
* Embed the words of the source sequences.
* Pack source sequences using [`pack_padded_sequence`](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pack_padded_sequence.html#torch.nn.utils.rnn.pack_padded_sequence). This converts padded source sequences into an object that can be processed by PyTorch recurrent units such as `nn.GRU` or `nn.LSTM`.
* Apply GRU computations to packed sequences obtained in the previous step
* Convert packed sequence of GRU outputs into padded representation with [`pad_packed_sequence`](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_packed_sequence.html#torch.nn.utils.rnn.pad_packed_sequence).

In [88]:
class Encoder(nn.Module):
    def __init__(self, src_dictionary_size, embed_size, hidden_size):
        """
        Args:
          src_dictionary_size: The number of words in the source dictionary.
          embed_size: The number of dimensions in the word embeddings.
          hidden_size: The number of features in the hidden state of GRU.
        """
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(src_dictionary_size, embed_size)
        self.gru = nn.GRU(input_size=embed_size, hidden_size=hidden_size)

    def forward(self, pad_seqs, seq_lengths, hidden):
        """
        Args:
          pad_seqs of shape (max_seq_length, batch_size): Padded source sequences.
          seq_lengths: List of sequence lengths.
          hidden of shape (1, batch_size, hidden_size): Initial states of the GRU.

        Returns:
          outputs of shape (max_seq_length, batch_size, hidden_size): Padded outputs of GRU at every step.
          hidden of shape (1, batch_size, hidden_size): Updated states of the GRU.
        """
        # YOUR CODE HERE
        # raise NotImplementedError()
  
        # Apply GRU computations to packed sequences obtained in the previous step
        # Convert packed sequence of GRU outputs into padded representation with [`pad_packed_sequence`]
        
        # Embed the words of the source sequences.
        y = self.embedding(pad_seqs)
        
        # Pack source sequences using [`pack_padded_sequence`]. 
        # This converts padded source sequences into an object that can be processed by PyTorch 
        # recurrent units such as `nn.GRU` or `nn.LSTM`.
        
        # rnn.pack_padded_sequence(input, lengths, batch_first=False, enforce_sorted=True)
        y = pack_padded_sequence(y, seq_lengths)
        
        # gru(input, h_0)
        # The input can also be a packed variable length sequence
        # h_0 contains the initial hidden state for the input sequence. 
        # Defaults to zeros if not provided
    
        # If a rnn.PackedSequence has been given as the input, 
        # the output will also be a packed sequence.
        # hidden containing the final hidden state for the input sequence
        outputs, hidden = self.gru(y, hidden)
        
        
        # Pads a packed batch of variable length sequences.
        # It is an inverse operation to pack_padded_sequence()
        # Returns:
        # Tuple of Tensor containing the padded sequence, and a Tensor 
        # containing the list of lengths of each sequence in the batch
        outputs, _ = pad_packed_sequence(outputs)
        return outputs, hidden
    
    def init_hidden(self, batch_size=1):
        return torch.zeros(1, batch_size, self.hidden_size)

In [89]:
def test_Encoder_shapes():
    hidden_size = 3
    encoder = Encoder(src_dictionary_size=5, embed_size=10, hidden_size=hidden_size)

    max_seq_length = 4
    batch_size = 2
    hidden = encoder.init_hidden(batch_size=batch_size)
    pad_seqs = torch.tensor([
        [        1,             2],
        [        2,     EOS_token],
        [        3, PADDING_VALUE],
        [EOS_token, PADDING_VALUE]
    ])

    outputs, new_hidden = encoder.forward(pad_seqs=pad_seqs, seq_lengths=[4, 2], hidden=hidden)
    assert outputs.shape == torch.Size([4, batch_size, hidden_size]), f"Bad outputs.shape: {outputs.shape}"
    assert new_hidden.shape == torch.Size([1, batch_size, hidden_size]), f"Bad new_hidden.shape: {new_hidden.shape}"
    print('Success')

test_Encoder_shapes()

Success


In [90]:
# This cell tests Encoder
import tests

def test_Encoder(Encoder):
    with torch.no_grad():
        net = Encoder(src_dictionary_size=5, embed_size=2, hidden_size=2)
        tests.set_weights_encoder(net)

        pad_seqs = torch.tensor([
            [1, 2],
            [2, 3],
            [3, 0],
            [4, 0]
        ])  # (max_seq_length, batch_size)
        seq_lengths = [4, 2]
        hidden = net.init_hidden(batch_size=2)

        outputs, new_hidden = net.forward(pad_seqs, seq_lengths, hidden)

        expected = torch.tensor([
            [ 0.0000, -0.0150],
            [ 0.0004, -0.0221],
            [ 0.0007, -0.0055],
            [ 0.0005,  0.0323]
        ])
        print('outputs[:, 0, :]:\n', outputs[:, 0, :])
        print('expected:\n', expected)
        assert torch.allclose(outputs[:,0,:], expected, atol=1e-4), "outputs do not match expected values"

        expected = torch.tensor([
            [ 0.0000, -0.0150],
            [ 0.0004, -0.0021]
        ])
        print('outputs[:2, 1, :]:\n', outputs[:2, 1, :])
        print('expected:\n', expected)
        assert torch.allclose(outputs[:2,1,:], expected, atol=1e-4), "outputs do not match expected values"

        expected = torch.tensor([[
            [ 0.0005,  0.0323],
            [ 0.0004, -0.0021]
        ]])
        print('new_hidden:\n', new_hidden)
        print('expected:\n', expected)
        assert torch.allclose(new_hidden, expected, atol=1e-4), "new_hidden does not match expected value"
        print('Success')

test_Encoder(Encoder)

outputs[:, 0, :]:
 tensor([[ 0.0000, -0.0150],
        [ 0.0004, -0.0221],
        [ 0.0007, -0.0055],
        [ 0.0005,  0.0323]])
expected:
 tensor([[ 0.0000, -0.0150],
        [ 0.0004, -0.0221],
        [ 0.0007, -0.0055],
        [ 0.0005,  0.0323]])
outputs[:2, 1, :]:
 tensor([[ 0.0000, -0.0150],
        [ 0.0004, -0.0021]])
expected:
 tensor([[ 0.0000, -0.0150],
        [ 0.0004, -0.0021]])
new_hidden:
 tensor([[[ 0.0005,  0.0323],
         [ 0.0004, -0.0021]]])
expected:
 tensor([[[ 0.0005,  0.0323],
         [ 0.0004, -0.0021]]])
Success


## Decoder

The decoder takes as input the representation computed by the encoder and transforms it into a sentence in the target language. The computational graph of the decoder is shown below:

<img src="seq2seq_decoder.png" width=500 align="top">

* $z_0$ is the output of the encoder, that is $z_0 = h_5$, thus `hidden_size` of the decoder should be the same as `hidden_size` of the encoder.
* $y_{i}$ are the log-probabilities of the words in the target language, the dimensionality of $y_{i}$ is the size of the target dictionary.
* $z_{i}$ is mapped to $y_{i}$ using a linear layer `self.out` followed by `F.log_softmax` (because we use `nn.NLLLoss` loss for training).
* Each cell of the decoder is a GRU, it receives as inputs the previous state $z_{i-1}$ and relu of the **embedding** of the previous word. Thus, you need to embed the words of the target language as well. The previous word is taken as the word with the maximum log-probability.

Note that the decoder outputs a word at every step and the same word is used as the input to the recurrent unit at the next step. At the beginning of decoding, the previous word input is fed with a special word SOS which stands for "start of a sentence". During training, we know the target sentence for decoding, therefore we can feed the correct words $y_i$ as inputs to the recurrent unit.

There is one extra thing that it is wise to take care of. When the target sentence is fed to the decoder during training, the decoder learns to generate only the next word (this scenario is called "teacher forcing"). At test time, the decoder works differently: it generates the whole sequence using its own predictions as inputs at each step. Therefore, it makes sense to train the decoder to produce full sentences. In order to do that, we will alternate between two modes during training:
* "teacher forcing": the decoder is fed with the words in the target sequence
* no "teacher forcing": the decoder generates the output sequence using its own predictions. In this case, we will generate sequences of the same length as the length of the longest sequence in `pad_tgt_seqs` (if `pad_tgt_seqs` is not `None`) or of length `MAX_LENGTH` (if `pad_tgt_seqs` is `None`).

You need to implement the decoder which has the structure shown in the figure above.

Notes:
* `SOS_token` is imported at the beginning of the notebook.
* **Running this code on GPU sometimes fails producing a CUDA error (if you know the reason, please let us know).** If this happens to you, please train the model on CPU.

In [91]:
class Decoder(nn.Module):
    def __init__(self, tgt_dictionary_size, embed_size, hidden_size):
        """
        Args:
          tgt_dictionary_size: The number of words in the target dictionary.
          embed_size: The number of dimensions in the word embeddings.
          hidden_size: The number of features in the hidden state.
        """
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(tgt_dictionary_size, embed_size)
        self.gru = nn.GRU(input_size=embed_size, hidden_size=hidden_size)
        self.out = nn.Linear(hidden_size, tgt_dictionary_size)

    def forward(self, hidden, pad_tgt_seqs=None, teacher_forcing=False):
        """
        Args:
          hidden of shape (1, batch_size, hidden_size): States of the GRU.
          pad_tgt_seqs of shape (max_out_seq_length, batch_size): Tensor of words (word indices) of the
              target sentence. If None, the output sequence is generated by feeding the decoder's outputs
              (teacher_forcing has to be False).
          teacher_forcing (bool): Whether to use teacher forcing or not.

        Returns:
          outputs of shape (max_out_seq_length, batch_size, tgt_dictionary_size): Tensor of log-probabilities
              of words in the target language.
          hidden of shape (1, batch_size, hidden_size): New states of the GRU.

        Note: Do not forget to transfer tensors that you may want to create in this function to the device
        specified by `hidden.device`.
        """
        if pad_tgt_seqs is None:
            assert not teacher_forcing, 'Cannot use teacher forcing without a target sequence.'
        #print("Hidden shape")
        #print(hidden.shape)
        batch_size = hidden.shape[1]
        hidden_size = hidden.shape[2]
        
        SOS = SOS_token * np.ones(batch_size).reshape(1,-1)
        SOS = torch.LongTensor(SOS) # SOS is of shape (1, batch_size)
        
        # In teacher forcing mode, the decoder is fed with the words in the target sequence
        if teacher_forcing:
            # returns a tensor of shape (max_out_seq_length - 1, batch_size) 
            # that contains all the target sequences in the batch, 
            # except the last token of each sequence. 
            # This is often used in Seq2Seq models, where the decoder inputs are 
            # typically the target sequences shifted one time step to the left, so that 
            # the model learns to predict the next token in the sequence based on the previous tokens. The last token of each target sequence is typically not used as input to the decoder, since there is no "next" token to predict after it.
            shifted_pad_tgt_seqs = pad_tgt_seqs[:-1]
            # concatenate the max_out_seq_length dimension withcommon batch_size
            y = torch.cat((SOS, shifted_pad_tgt_seqs), dim=0)
            y = self.embedding(y)
            
            # torch.relu is a functional interface to the ReLU activation function. 
            # It applies the element-wise ReLU function to the input tensor and returns the 
            # result as a new tensor. torch.relu is a stateless operation, which means that 
            # it does not have any trainable parameters.

            # nn.ReLU is a module that implements the ReLU activation function as a layer 
            # in a neural network. It can be used as a building block in a neural network, 
            # and can be trained along with the other layers of the network. nn.ReLU is a 
            # stateful operation, which means that it has trainable parameters 
            # (i.e., the weights and biases of the layer).
            y = torch.relu(y)
            outputs, hidden = self.gru(y, hidden)
            outputs = F.log_softmax(self.out(outputs), dim=2)
        
        else:
            
            







            
            # At the beginning of decoding, the previous word input is fed with a special word 
            # SOS which stands for "start of a sentence". 
            # y0 = SOS
            y = SOS
            max_length = MAX_LENGTH if pad_tgt_seqs is None else len(pad_tgt_seqs)
            for i in range(max_length):
                # Each cell of the decoder is a GRU, 
                # it receives as inputs the previous state $z_{i-1}$ and 
                # relu of the **embedding** of the previous word. 

                # During training, we know the target sentence for decoding, 
                # therefore we can feed the correct words $y_i$ as inputs to the recurrent unit.
                y = torch.LongTensor(y)
                y = self.embedding(y)
                y = torch.relu(y)
                
                # $z_0$ is the output of the encoder, that is $z_0 = h_5$, 
                # thus `hidden_size` of the decoder should be the same as `hidden_size` of the encoder.
                # $z_{i}$ is mapped to $y_{i}$ using a linear layer `self.out` 
                # followed by `F.log_softmax` 
                # (because we use `nn.NLLLoss` loss for training).
                z, hidden = self.gru(y, hidden)
                z = self.out(z)
                # $y_{i}$ are the log-probabilities of the words in the target language, 
                # the dimensionality of $y_{i}$ is the size of the target dictionary.
                y = F.log_softmax(z, dim=2)
                # concatenate the probabilities of the target words 
                # Note that the decoder outputs a word at every step and 
                # thus, you need to embed the words of the target language as well. 
                outputs = y if i == 0 else torch.cat((outputs,y), dim=0)
                # The previous word is taken as the word with the maximum log-probability.
                y = torch.argmax(y, dim=2)
        
        # Do not forget to transfer tensors that you may want to create in this function 
        # to the device specified by `hidden.device`
        outputs.to(hidden.device)
        hidden.to(hidden.device)
        # YOUR CODE HERE
        # raise NotImplementedError()
        return outputs, hidden

In [92]:
def test_Decoder_shapes():
    hidden_size = 2
    tgt_dictionary_size = 5
    test_decoder = Decoder(tgt_dictionary_size, embed_size=10, hidden_size=hidden_size)

    max_seq_length = 4
    batch_size = 2
    pad_tgt_seqs = torch.tensor([
        [        1,             2],
        [        2,     EOS_token],
        [        3, PADDING_VALUE],
        [EOS_token, PADDING_VALUE]
    ])  # [max_seq_length, batch_size]

    hidden = torch.zeros(1, batch_size, hidden_size)
    outputs, new_hidden = test_decoder.forward(hidden, pad_tgt_seqs, teacher_forcing=False)

    assert outputs.size(0) <= 4, f"Too long output sequence: outputs.size(0)={outputs.size(0)}"
    assert outputs.shape[1:] == torch.Size([batch_size, tgt_dictionary_size]), \
        f"Bad outputs.shape[1:]={outputs.shape[1:]}"
    assert new_hidden.shape == torch.Size([1, batch_size, hidden_size]), f"Bad new_hidden.shape={new_hidden.shape}"

    outputs, new_hidden = test_decoder.forward(hidden, pad_tgt_seqs, teacher_forcing=True)
    assert outputs.shape == torch.Size([4, batch_size, tgt_dictionary_size]), \
        f"Bad shape outputs.shape={outputs.shape}"
    assert new_hidden.shape == torch.Size([1, batch_size, hidden_size]), f"Bad new_hidden.shape={new_hidden.shape}"

    # Generation mode
    outputs, new_hidden = test_decoder.forward(hidden, None, teacher_forcing=False)
    assert outputs.shape[1:] == torch.Size([batch_size, tgt_dictionary_size]), \
        f"Bad outputs.shape[1:]={outputs.shape[1:]}"
    assert new_hidden.shape == torch.Size([1, batch_size, hidden_size]), f"Bad new_hidden.shape={new_hidden.shape}"

    print('Success')

test_Decoder_shapes()

Success


In [93]:
# This cell tests Decoder
def test_Decoder_no_forcing(Decoder):
    # Test without teaching_forcing
    with torch.no_grad():
        net = Decoder(tgt_dictionary_size=5, embed_size=2, hidden_size=2)
        tests.set_weights_decoder(net)

        pad_target_seqs = torch.tensor([
            [1, 2],
            [2, 3],
            [3, 0],
            [4, 0]
        ])

        hidden = torch.tensor([
            [1., -1.],
            [1., -1.],
        ]).view(1, 2, 2)
        outputs, new_hidden = net.forward(hidden, pad_target_seqs, teacher_forcing=False)
        outputs, new_hidden = outputs.float(), new_hidden.float()

        expected = torch.tensor([
            [-1.3788, -1.3503, -1.5045, -2.1493, -1.8954],
            [-1.5265, -1.4880, -1.5090, -1.8655, -1.7096],
            [-1.6097, -1.5715, -1.5319, -1.7189, -1.6249],
            [-1.6317, -1.6037, -1.5593, -1.6543, -1.6007]
        ])

        print('outputs[:, 0, :]:\n', outputs[:, 0, :])
        print('expected:\n', expected)
        assert torch.allclose(outputs[:,0,:], expected, atol=1e-4), "outputs do not match expected values"

        print('outputs[:, 1, :]:\n', outputs[:, 1, :])
        print('expected:\n', expected)
        assert torch.allclose(outputs[:,1,:], expected, atol=1e-4), "outputs do not match expected values"

        print('Success')

test_Decoder_no_forcing(Decoder)

outputs[:, 0, :]:
 tensor([[-1.3788, -1.3503, -1.5045, -2.1493, -1.8954],
        [-1.5265, -1.4880, -1.5090, -1.8655, -1.7096],
        [-1.6097, -1.5715, -1.5319, -1.7189, -1.6249],
        [-1.6317, -1.6037, -1.5593, -1.6543, -1.6007]])
expected:
 tensor([[-1.3788, -1.3503, -1.5045, -2.1493, -1.8954],
        [-1.5265, -1.4880, -1.5090, -1.8655, -1.7096],
        [-1.6097, -1.5715, -1.5319, -1.7189, -1.6249],
        [-1.6317, -1.6037, -1.5593, -1.6543, -1.6007]])
outputs[:, 1, :]:
 tensor([[-1.3788, -1.3503, -1.5045, -2.1493, -1.8954],
        [-1.5265, -1.4880, -1.5090, -1.8655, -1.7096],
        [-1.6097, -1.5715, -1.5319, -1.7189, -1.6249],
        [-1.6317, -1.6037, -1.5593, -1.6543, -1.6007]])
expected:
 tensor([[-1.3788, -1.3503, -1.5045, -2.1493, -1.8954],
        [-1.5265, -1.4880, -1.5090, -1.8655, -1.7096],
        [-1.6097, -1.5715, -1.5319, -1.7189, -1.6249],
        [-1.6317, -1.6037, -1.5593, -1.6543, -1.6007]])
Success


In [94]:
# This cell tests Decoder
def test_Decoder_with_forcing(Decoder):
    # Test with teaching_forcing
    with torch.no_grad():
        net = Decoder(tgt_dictionary_size=5, embed_size=2, hidden_size=2)
        tests.set_weights_decoder(net)

        pad_target_seqs = torch.tensor([
            [1, 2],
            [2, 3],
            [3, 0],
            [4, 0]
        ])

        hidden = torch.tensor([
            [1., -1.],
            [1., -1.],
        ]).view(1, 2, 2)
        outputs, new_hidden = net.forward(hidden, pad_target_seqs, teacher_forcing=True)
        outputs, new_hidden = outputs.float(), new_hidden.float()

        expected = torch.tensor([
            [-1.3788, -1.3503, -1.5045, -2.1493, -1.8954],
            [-1.5265, -1.4880, -1.5090, -1.8655, -1.7096],
            [-1.5906, -1.5591, -1.5397, -1.7299, -1.6393],
            [-1.6109, -1.5899, -1.5668, -1.6664, -1.6159]
        ])
        print('outputs[:, 0, :]:\n', outputs[:, 0, :])
        print('expected:\n', expected)
        assert torch.allclose(outputs[:,0,:], expected, atol=1e-4), "outputs do not match expected values"

        expected = torch.tensor([
            [-1.3788, -1.3503, -1.5045, -2.1493, -1.8954],
            [-1.5091, -1.4770, -1.5174, -1.8772, -1.7245],
            [-1.5706, -1.5460, -1.5476, -1.7424, -1.6548],
            [-1.6209, -1.5961, -1.5623, -1.6618, -1.6087]
        ])
        print('outputs[:, 1, :]:\n', outputs[:, 1, :])
        print('expected:\n', expected)
        assert torch.allclose(outputs[:,1,:], expected, atol=1e-4), "outputs do not match expected values"

        print('Success')

test_Decoder_with_forcing(Decoder)

outputs[:, 0, :]:
 tensor([[-1.3788, -1.3503, -1.5045, -2.1493, -1.8954],
        [-1.5265, -1.4880, -1.5090, -1.8655, -1.7096],
        [-1.5906, -1.5591, -1.5397, -1.7299, -1.6393],
        [-1.6109, -1.5899, -1.5668, -1.6664, -1.6159]])
expected:
 tensor([[-1.3788, -1.3503, -1.5045, -2.1493, -1.8954],
        [-1.5265, -1.4880, -1.5090, -1.8655, -1.7096],
        [-1.5906, -1.5591, -1.5397, -1.7299, -1.6393],
        [-1.6109, -1.5899, -1.5668, -1.6664, -1.6159]])
outputs[:, 1, :]:
 tensor([[-1.3788, -1.3503, -1.5045, -2.1493, -1.8954],
        [-1.5091, -1.4770, -1.5174, -1.8772, -1.7245],
        [-1.5706, -1.5460, -1.5476, -1.7424, -1.6548],
        [-1.6209, -1.5961, -1.5623, -1.6618, -1.6087]])
expected:
 tensor([[-1.3788, -1.3503, -1.5045, -2.1493, -1.8954],
        [-1.5091, -1.4770, -1.5174, -1.8772, -1.7245],
        [-1.5706, -1.5460, -1.5476, -1.7424, -1.6548],
        [-1.6209, -1.5961, -1.5623, -1.6618, -1.6087]])
Success


In [95]:
# This cell tests Decoder
def test_Decoder_generation(Decoder):
    # Test in generation mode
    with torch.no_grad():
        net = Decoder(tgt_dictionary_size=5, embed_size=2, hidden_size=2)
        tests.set_weights_decoder(net)

        hidden = torch.tensor([
            [1., -1.],
            [1., -1.],
        ]).view(1, 2, 2)
        outputs, new_hidden = net.forward(hidden, None, teacher_forcing=False)
        outputs, new_hidden = outputs.float(), new_hidden.float()

        expected = torch.tensor([
            [-1.3788, -1.3503, -1.5045, -2.1493, -1.8954],
            [-1.5265, -1.4880, -1.5090, -1.8655, -1.7096],
            [-1.6097, -1.5715, -1.5319, -1.7189, -1.6249],
            [-1.6317, -1.6037, -1.5593, -1.6543, -1.6007],
            [-1.6417, -1.6201, -1.5757, -1.6212, -1.5899],
            [-1.6459, -1.6282, -1.5851, -1.6042, -1.5852],
            [-1.6476, -1.6321, -1.5904, -1.5956, -1.5831],
            [-1.6541, -1.6379, -1.5912, -1.5881, -1.5782],
            [-1.6571, -1.6408, -1.5919, -1.5842, -1.5759],
            [-1.6585, -1.6421, -1.5924, -1.5822, -1.5748]
        ])
        print('outputs[:, 0, :]:\n', outputs[:, 0, :])
        print('expected:\n', expected)
        assert torch.allclose(outputs[:,0,:], expected, atol=1e-4), "outputs do not match expected values"

        print('outputs[:, 1, :]:\n', outputs[:, 1, :])
        print('expected:\n', expected)
        assert torch.allclose(outputs[:,1,:], expected, atol=1e-4), "outputs do not match expected values"

        print('Success')

test_Decoder_generation(Decoder)

outputs[:, 0, :]:
 tensor([[-1.3788, -1.3503, -1.5045, -2.1493, -1.8954],
        [-1.5265, -1.4880, -1.5090, -1.8655, -1.7096],
        [-1.6097, -1.5715, -1.5319, -1.7189, -1.6249],
        [-1.6317, -1.6037, -1.5593, -1.6543, -1.6007],
        [-1.6417, -1.6201, -1.5757, -1.6212, -1.5899],
        [-1.6459, -1.6282, -1.5851, -1.6042, -1.5852],
        [-1.6476, -1.6321, -1.5904, -1.5956, -1.5831],
        [-1.6541, -1.6379, -1.5912, -1.5881, -1.5782],
        [-1.6571, -1.6408, -1.5919, -1.5842, -1.5759],
        [-1.6585, -1.6421, -1.5924, -1.5822, -1.5748]])
expected:
 tensor([[-1.3788, -1.3503, -1.5045, -2.1493, -1.8954],
        [-1.5265, -1.4880, -1.5090, -1.8655, -1.7096],
        [-1.6097, -1.5715, -1.5319, -1.7189, -1.6249],
        [-1.6317, -1.6037, -1.5593, -1.6543, -1.6007],
        [-1.6417, -1.6201, -1.5757, -1.6212, -1.5899],
        [-1.6459, -1.6282, -1.5851, -1.6042, -1.5852],
        [-1.6476, -1.6321, -1.5904, -1.5956, -1.5831],
        [-1.6541, -1.6379, -1.5912

## Training of sequence-to-sequence model using mini-batches

Now we are going to train the sequence-to-sequence model on the toy translation dataset.

In [96]:
# Create the seq2seq model
hidden_size = embed_size = 256
encoder = Encoder(trainset.input_lang.n_words, embed_size, hidden_size).to(device)
decoder = Decoder(trainset.output_lang.n_words, embed_size, hidden_size).to(device)

In [97]:
teacher_forcing_ratio = 0.5

Implement the training loop in the cell below. In the training loop, we first encode source sequences using the encoder, then we decode the encoded state using the decoder. The decoder outputs log-probabilities of words in the target language. We need to use these log-probabilities and the indexes of the words in the target sequences to compute the loss.

The loss is
\begin{align*}
L = - \frac{1}{N} \sum_{n} \sum_{t=1}^{T_n}
\log p\left(\mathbf{y}_t^{(n)} \:\Bigl|\: \mathbf{y}_{<t}^{(n)}, \mathbf{X}^{(n)} \right)
\end{align*}
where $T_n$ is the length of the $n$-th target sequence and $N= \sum_{n=1} T_n$ is the total number of words in all the sentences of the mini-batch.

Recommended hyperparameters:
- Encoder optimizer: Adam with learning rate 0.001
- Decoder optimizer: Adam with learning rate 0.001
- Number of epochs: 30
- Toggle `teacher_forcing` on and off (for each mini-batch) according to the `teacher_forcing_ratio` specified above.

Hints:
- Training should proceed relatively fast.
- If you do well, the training loss should reach 0.1 in 30 epochs.
- Slight overlearning may happen (you can see that if you track the test error during training) but you can ignore this problem. 
- **Important:** When computing the loss, you need to ignore the padded values. This can easily be done by using argument `ignore_index` of function [`nll_loss`](
https://pytorch.org/docs/stable/nn.functional.html#torch.nn.functional.nll_loss).

In [98]:
if not skip_training:
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    # Encoder optimizer: Adam with learning rate 0.001
    encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr = 0.001)
    # Decoder optimizer: Adam with learning rate 0.001
    decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr = 0.001)
    
    mean_training_errors = []
    
    # Number of epochs: 30
    epochs = 30
    
    for epoch in range(epochs):
        training_errors = []
        for src_sentence, length, tgt_sentence in trainloader:
            encoder_optimizer.zero_grad()
            hidden = encoder.init_hidden(len(length))
            # In the training loop, we first encode source sequences using the encoder
            _, hidden = encoder.forward(src_sentence,length,hidden)
            
            decoder_optimizer.zero_grad()
            
            # Toggle teacher_forcing on and off (for each mini-batch) according to the teacher_forcing_ratio specified above
            teacher_forcing = np.random.randn() < teacher_forcing_ratio
            
            # then we decode the encoded state using the decoder. 
            # The decoder outputs log-probabilities of words in the target language. 
            # We need to use these log-probabilities and the indexes of the words in 
            # the target sequences to compute the loss.
            outputs, _ = decoder.forward(hidden,tgt_sentence,teacher_forcing)
            outputs = torch.flatten(outputs,end_dim=1)
            
            # When computing the loss, you need to ignore the padded values. 
            # This can easily be done by using argument ignore_index of function nll_loss
            loss = F.nll_loss(outputs,tgt_sentence.view(-1),ignore_index=PADDING_VALUE)
            
            training_errors.append(loss.item())
            loss.backward()
            encoder_optimizer.step()
            decoder_optimizer.step()
            
        mean_training_errors.append(np.mean(training_errors))
        print(f"Epoch {epoch} mean training error: {mean_training_errors[epoch]}")

In [99]:
# Save the model to disk (the pth-files will be submitted automatically together with your notebook)
# Set confirm=False if you do not want to be asked for confirmation before saving.
if not skip_training:
    tools.save_model(encoder, '1_rnn_encoder.pth', confirm=False)
    tools.save_model(decoder, '1_rnn_decoder.pth', confirm=False)

In [100]:
if skip_training:
    hidden_size = 256
    encoder = Encoder(trainset.input_lang.n_words, embed_size, hidden_size)
    tools.load_model(encoder, '1_rnn_encoder.pth', device)
    
    decoder = Decoder(trainset.output_lang.n_words, embed_size, hidden_size)
    tools.load_model(decoder, '1_rnn_decoder.pth', device)

Model loaded from 1_rnn_encoder.pth.
Model loaded from 1_rnn_decoder.pth.


In [101]:
# This cell tests training accuracy

## Evaluation

Next we need to implement a function that converts input sequences to output sequences using the trained sequence-to-sequence model.

Notes:
* Since we do not need to compute the gradients in the evaluation phase, we can speed up the computations by using the statement `with torch.no_grad():`.
* Please transfer the tensors to `device` inside this function.

In [102]:
def translate(encoder, decoder, pad_src_seqs, src_seq_lengths):
    """Translate sequences from the source language to the target language using the trained model.
    
    Args:
      encoder (Encoder): Trained encoder.
      decoder (Decoder): Trained decoder.
      pad_src_seqs of shape (max_src_seq_length, batch_size): Padded source sequences.
      src_seq_lengths: List of source sequence lengths.
    
    Returns:
      out_seqs of shape (MAX_LENGTH, batch_size): LongTensor of word indices of the output sequences.
    """
    # YOUR CODE HERE
    # raise NotImplementedError()
    torch.no_grad()
    hidden = encoder.init_hidden(len(src_seq_lengths))
    _, hidden = encoder.forward(pad_src_seqs,src_seq_lengths,hidden)
    out_seqs, hidden = decoder.forward(hidden,None,False)
    out_seqs = torch.argmax(out_seqs,dim=2)
    return out_seqs

In [103]:
def test_translate_shapes():
    pad_src_seqs = torch.tensor([
        [1, 2],
        [2, 3],
        [3, 0],
        [4, 0]
    ])

    out_seqs = translate(encoder, decoder, pad_src_seqs, src_seq_lengths=[4, 2])
    assert out_seqs.shape == torch.Size([MAX_LENGTH, 2]), f"Wrong out_seqs.shape: {out_seqs.shape}"
    print('Success')

test_translate_shapes()

Success


Let us now translate a few sentences from the training set and print the source, target, and produced output.

If you trained the model well enough, the model should memorize the training data well.

In [104]:
def seq_to_tokens(seq, lang):
    'Convert a sequence of word indices into a list of words (strings).'
    sentence = []
    for i in seq:
        if i == EOS_token:
            break
        sentence.append(lang.index2word[i.item()])
    return(sentence)

def seq_to_string(seq, lang):
    'Convert a sequence of word indices into a sentence string.'
    return(' '.join(seq_to_tokens(seq, lang)))

In [105]:
# Translate a few sentences from the training set
print('Translate training data:')
print('-----------------------------')
pad_src_seqs, src_seq_lengths, pad_tgt_seqs = next(iter(trainloader))
out_seqs = translate(encoder, decoder, pad_src_seqs, src_seq_lengths)

for i in range(5):
    print('SRC:', seq_to_string(pad_src_seqs[:,i], trainset.input_lang))
    print('TGT:', seq_to_string(pad_tgt_seqs[:,i], trainset.output_lang))
    print('OUT:', seq_to_string(out_seqs[:,i], trainset.output_lang))
    print('')

Translate training data:
-----------------------------
SRC: je suis en train d attendre le bus .
TGT: i m waiting for the bus .
OUT: i m waiting for the bus .

SRC: ce sont celles qui veulent s y rendre .
TGT: they are the ones who want to go .
OUT: they are the ones who want to go .

SRC: je suis le premier musicien de la famille .
TGT: i am the first musician in my family .
OUT: i am the first musician in my family .

SRC: je ne suis pas en mal d argent .
TGT: i m not pressed for money .
OUT: i m not pressed for money .

SRC: elle est accoutumee a veiller toute la nuit .
TGT: she is used to staying up all night .
OUT: she is used to staying up all night .



Now we translate random sentences from the test set. A well-trained model should output sentences that look similar to the target ones. The mistakes are usually done for words that were rare in the training set.

In [106]:
testset = TranslationDataset(data_dir, train=False)
testloader = DataLoader(dataset=testset, batch_size=64, shuffle=False, collate_fn=collate)

In [107]:
print('Translate test data:')
print('-----------------------------')
pad_src_seqs, src_seq_lengths, pad_tgt_seqs = next(iter(testloader))
out_seqs = translate(encoder, decoder, pad_src_seqs, src_seq_lengths)

for i in range(5):
    print('SRC:', seq_to_string(pad_src_seqs[:,i], testset.input_lang))
    print('TGT:', seq_to_string(pad_tgt_seqs[:,i], testset.output_lang))
    print('OUT:', seq_to_string(out_seqs[:,i], testset.output_lang))
    print('')

Translate test data:
-----------------------------
SRC: il est toujours en train de se plaindre .
TGT: he is constantly complaining .
OUT: he is walking complaining .

SRC: je suis ravi que vous ayez souleve ca .
TGT: i m glad you brought that up .
OUT: i m glad you brought that up .

SRC: elle est entierement devouee a ses trois enfants .
TGT: she is devoted to her three children .
OUT: she is anxious to speak up with .

SRC: je suis ravi que tu aies souleve ca .
TGT: i m glad you brought that up .
OUT: i m glad you brought that up .

SRC: tu n es pas aussi maligne que moi .
TGT: you re not as smart as me .
OUT: you re not as smart as me .



## Compute BLEU score

Let us now compute the [BLEU score](https://en.wikipedia.org/wiki/BLEU) for the translations produced by our model. We can use the PyTorch function [bleu_score](https://pytorch.org/text/stable/data_metrics.html?highlight=bleu_score#torchtext.data.metrics.bleu_score) for that.

* **Your model should achieve a minimum BLEU score of 90 on the training set.**
* The BLEU score on the test set should be greater than 40.

The model can severly overfit to the training set and we do not cope with the overfitting problem in this exercise.

In [108]:
from torchtext.data.metrics import bleu_score

In [109]:
# Create translations for the training set
candidate_corpus = []
references_corpus = []
for pad_src_seqs, src_seq_lengths, pad_tgt_seqs in trainloader:
    out_seqs = translate(encoder, decoder, pad_src_seqs, src_seq_lengths)
    candidate_corpus.extend([seq_to_tokens(seq, trainset.output_lang) for seq in out_seqs.T])
    references_corpus.extend([[seq_to_tokens(seq, trainset.output_lang)] for seq in pad_tgt_seqs.T])

# Compute BLEU for translations
score = bleu_score(candidate_corpus, references_corpus)
print(f'BLEU score on training data: {score*100}')
assert score*100 > 90, "The BLEU score is too low."

BLEU score on training data: 96.95085883140564


In [110]:
# Create translations for the test set
candidate_corpus = []
references_corpus = []
for pad_src_seqs, src_seq_lengths, pad_tgt_seqs in testloader:
    out_seqs = translate(encoder, decoder, pad_src_seqs, src_seq_lengths)
    candidate_corpus.extend([seq_to_tokens(seq, testset.output_lang) for seq in out_seqs.T])
    references_corpus.extend([[seq_to_tokens(seq, testset.output_lang)] for seq in pad_tgt_seqs.T])

# Compute BLEU for translations
score = bleu_score(candidate_corpus, references_corpus)
print(f'BLEU score on test data: {score*100}')
assert score*100 > 40, "The BLEU score is too low."

BLEU score on test data: 47.462530897787516


<div class="alert alert-block alert-info">
<b>Conclusion</b>
</div>

In this notebook:
* We learned how recurrent neural networks can be used to build a sequence-to-sequence model.
* We trained a sequence-to-sequence model for statistical machine translation.